# 白话“朴素贝叶斯”

+ 朴素贝叶斯（Naive Bayers）是一种基于概率统计的**分类**方法，“朴素贝叶斯”预测的是概率，是一个生成模型；
+ 应用：本文分类、文本情感预测；
+ 朴素贝叶斯是最简单的概率有向图模型。

## 为什么称之为“朴素”？

+ 引入特征**条件独立性假设**，即在类别确定的前提下，数据的各个特征是独立的；

说明：一般情况下，一个数据的各个特征是有关系的，例如一个人的身高和体重，但是朴素贝叶斯假设它们是独立的，虽然有悖于我们的认知，但是基于这个假设，在很多问题上，都能有很好的效果。

## 用概率论中的乘法公式理解贝叶斯定理（Bayes theorem）

在本科《概率论与数理统计》的教材上，你会看到下面的公式：

$$
P(B_i|A) = \frac{P(B_i)P(A|B_i)}{\sum_{j=1}^{n}P(B_j)P(A|B_j)}
$$

但其实，你要你懂得条件概率与联合概率，用乘法公式就能够自己推出贝叶斯公式，上面右边式子中，分母是全概率公式，与本文无关，可以忽略。

### 自己推导朴素贝叶斯公式

随机变量 $X$ 和 $Y$ 同时发生的概率 $P(X,Y)$ 可以写成：

$$
P(X,Y) = P(X)P(Y|X)
$$

也可以写成：

$$
P(X,Y) = P(Y)P(X|Y)
$$

把它们合在一起，写成：
$$
P(X)P(Y|X)= P(Y)P(X|Y)
$$

于是就有：
$$
P(Y|X) = \cfrac{P(Y)P(X|Y)}{P(X)}
$$

我们看到，贝叶斯公式就用高中学习过的概率论的乘法公式，就可以推导出来。

### 解读贝叶斯公式

+ $X$ 是输入，一般而言是一个向量，这个向量的每个分量表示单独的一个特征，$P(X)$ 与 $Y$ 无关，放在分母可以认为是后验概率的归一化因子，这部分对于所有的数据都是一样的，所以我们可以不用计算它；
+ $Y$ 是输出，是一个类别变量，是标量，$P(Y)$ 是先验概率，即不知道任何信息的时候，我们对于一个类别变量的概率的估计，如果不是很能理解“先验概率”这个定义，可以参考本科《概率论与数理统计》教材相关章节，即相对于知道了某些信息的后验概率 $P(Y|X)$ ，什么都不知道的时候 $P(Y)$ 就是先验概率；
+ $P(X|Y)$ 是条件概率，是在类别确定的时候，出现特征 $X$ 的概率，特征的**条件独立性假设就应用在这里；

假设一个数据 $X$ 有 $3$ 个特征，即 $X = (x_1,x_2,x_3)^T$，那么在类别 $y$ 确定的条件下，数据 $X$ 的各个特征相互独立，用公式写出来就是：

$$
P(X|y) = P(x_1,x_2,x_3|y) = P(x_1|y) P(x_2|y) P(x_3|y) 
$$


为什么要引入这个假设呢？如果你使用贝叶斯定理去做一次垃圾短信分类或者垃圾邮件分类的任务，你就会发现 $P(x_1,x_2,x_3|y)$ 不好算，因为要判断 $3$ 个特征是否同时出现在一个数据里，甚至 $P(x_1,x_2,x_3|y)$ 这个值是很小的，引入条件独立性假设是为了方便计算，因为单独的特征的条件概率 $P(x_1|y)$、$P(x_2|y)$、$P(x_3|y) $ 好算，只用计数就可以了。

+ $P(Y|X)$ 是一个条件概率，我们这里称之为**后验概率**，即知道了一些信息以后认为属于某一类的概率，后验概率是我们要学习的。


李航《统计学习方法》P7：

> 由条件概率表示的模型为概率模型。



所以朴素贝叶斯分类法就是从训练数据集中，计算类别的先验分布 $P(Y)$ 和条件概率分布 $P(X|Y)$，然后就可以计算出联合概率分布  $P(X,Y)=P(Y)P(X|Y)$，**因为分母一样大，仅仅计算出贝叶斯公式分子的部分，然后比较分子的大小就可以了**，联合概率大的，后验概率肯定也大。

选择后验概率最大的类作为 $x$ 的输出，即每一个类别的概率都要算一遍，然后取概率最大的类作为输出：

$$
y = f(x) = \arg \max_{c_k} P(Y=c_k)\prod_{j}P(X^{(j)}=x^{(j)}|Y=c_k)
$$

## 应用贝叶斯公式的问题

具体在工程中，会遇到两个问题：

问题1：计算概率的乘积会使得很多小数连乘，最后得到的数值越来越小；
解决1：因此我们通常对上式取对数（以 $e$ 为底，其它大于 $1$ 的数为底都行），取对数的好处是：1、对最终结果没有影响，原来大的数，取对数以后仍然大； 2、取对数以后，连乘变连加。

问题2：有的特征的取值，在某些特定的类别上并不出现，即会出现了概率为 $0$ 的情况。例如：在“非垃圾邮件中”，不会出现单词“国务院”。这时就要使用贝叶斯估计了：我们常常取 $\lambda=1$ ，在分子分母同时做加法，使得分子分母都不为 $0$，并且满足加和为 1。这一步也叫拉普拉斯修正（Laplacian correction）。

1、先验概率的贝叶斯估计

$$
P_{\lambda} (Y=c_k) = \frac{\sum_{i=1}^{N}I(y_i=c_k)+\lambda}{N+K\lambda}
$$

数数字，用样本概率估计总体概率的方法叫做极大似然估计，即我们通过样本得到的频率作为概率的估计，下面的“条件概率的贝叶斯估计”也是如此。

2、条件概率的贝叶斯估计

$$
P_{\lambda} (X^{(j)}=a_{jl}|Y=c_k) = \frac{\sum_{i=1}^{N}I(x_i^{(j)}=a_{jl},y_i=c_k)+\lambda}{\sum_{i=1}^{N} I(y_i=c_k)+S_j\lambda}
$$

$S_j$ 表示第 $j$ 个特征 unique 以后得到的数，即第 $j$ 个特征的不同取值的个数。**满足非负性和归一性，因此是概率分布**。

说明：1、上面做概率估计的方法是极大似然估计（其实就是数数字）和贝叶斯估计（在数数字的基础上加上了拉普拉斯平滑）；
2、比起原来的数数字的方法而言，分子都加上 $1$，分母都加上的是这个类别的个数。李航《统计学习方法》 P52 例2 是一个很好的例子。

## “后验概率最大化”等价于“期望风险最小化”

李航的《统计学习方法》P48：

> 朴素贝叶斯算法将实例分到“后验概率最大”的类中，这等价于期望风险最小化。

后验概率最大化的含义：后验概率最大化等价于 0-1 损失函数时的期望风险最小化。定义 0-1 损失函数，期望是对联合概率分布 $P(X,Y)$ 取的。期望风险是损失函数对于联合概率分布的期望。

说明：“后验概率最大化”符合机器学习的一般套路：定义损失函数，找一个<span class="burk">决策函数</span>或者<span class="burk">条件概率分布</span>使得损失最小的那个决策函数或者条件概率分布就是最好的模型，朴素贝叶斯找的就是使得“期望风险”最小的条件概率分布。


## 朴素贝叶斯方法实际上学习到的是生成数据的机制

+ 生成模型和判别模型，其实我一开始根本搞不清楚，后来我发现，只要记住一点：生成模型首先是概率模型，首先要计算概率，但不是直接计算后验概率，得通过先计算联合概率，然后比较联合概率的大小，这是生成模型；

+ 我们的朴素贝叶斯分类法就是先计算联合概率分布，根据贝叶斯公式，联合概率分布大的后验概率也大；

+ 典型的判别模型，同样也是计算概率的有：逻辑回归模型、决策树模型，但它们压根没有联合概率什么事，是直接计算 $P(Y|X)$ 的。

关于判别模型和生成模型，可以参考李航《统计学习方法》第 12 章统计学习方法总结。


## 参考资料

1、李航《统计学习方法》第 4 章 朴素贝叶斯法

2、刘建平《scikit-learn 朴素贝叶斯类库使用小结》
https://www.cnblogs.com/pinard/p/6074222.html

3、scikit-learn 官方文档
http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

4、朴素贝叶斯理论推导与三种常见模型
http://wepon.me/2015/09/09/naive-bayes/
说明：wepon 据说是个大神，他的文章要好好看看。


5、阮一峰也写了文章：
朴素贝叶斯分类器的应用
http://www.ruanyifeng.com/blog/2013/12/naive_bayes_classifier.html

6、张洋：算法杂货铺——分类算法之朴素贝叶斯分类(Naive Bayesian classification)
http://www.cnblogs.com/leoo2sk/archive/2010/09/17/1829190.html



7、我买的书黄永昌的博客：

零基础学习朴素贝叶斯算法
http://blog.kamidox.com/naive-bayes.html

黄永昌的简书：
https://www.jianshu.com/p/770a315d818d?utm_campaign=maleskine&utm_content=note&utm_medium=seo_notes&utm_source=recommendation

